In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import csv

In [ ]:
# Try scraping with BeautifulSoup only, not Splinter
# executable_path = {'executable_path': '/Users/Bons/Downloads/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# List of pages to be looped through
url_list = []

# URL of page to be scraped
for i in range(1,50):
    url = 'https://slickdeals.net/?page=' + str(i)
    url_list.append(url)
#     print(url_list)

In [ ]:
sale_items = []

# Trying to loop through pages 1-50 of Slickdeals.net
for url in url_list:
    
    response = requests.get(url)
    soup = bs(response.text, 'html.parser')
    
    results = soup.find_all('div')
    
    #Create a dataframe to hold
    
    for result in results:
    # Error handling
        try:
            # Identify and return title of listing
            title = result.find('div', class_="priceLine")
            # Identify and return original price of listing
            old_price = result.find('span', class_="oldListPrice")
#             old_price = old_price.text
            # Identify and return new price of listing
            new_price = result.find('div', class_="itemPrice")
#             new_price = new_price.text.strip()
            # Print results only if title,a price, and link are available
            if (title and old_price and new_price):
                print('-------------')
                print("Title")
                print(title["title"])
                print("Old Price")
                print(old_price.text)
                print("New Price")
                print(new_price.text.strip())
                sale_items.append({"Item" : title["title"], "Old Price" : old_price.text, "Sale Price": new_price.text.strip()})
        except AttributeError as e:
            print(e)
            
#     browser.click_link_by_partial_text('Next')

In [ ]:
sale_items_df = pd.DataFrame(sale_items)
sale_items_df

In [ ]:
sale_items_df.append({"Store" : "Slickdeals.net"})

In [ ]:
sale_items_df.to_csv("slickdeals.csv", index=True)

In [ ]:
master_list_df = sale_items_df
# master_list_df.columns
# pick_up_method.columns

In [ ]:
# Split item column after "free"

# leaves "free shipping" and "store pick up"
pick_up_method = pd.DataFrame(sale_items_df.Item.str.split('+').str[1])
pick_up_method.to_csv('slickdeal_items_w_pickup.csv', index=True)



In [ ]:
#drops the shipping/store pickup
split_items = pd.DataFrame(sale_items_df.Item.str.split('+').str[0])
split_items.to_csv("slickdeal_items.csv", index=True)

In [ ]:
merged_items_df = split_items.merge(pick_up_method, left_index=True, right_index=True, how='inner')

In [ ]:
merged_items_df = merged_items_df.rename(columns = {"Item_x" : "Item",
                                  "Item_y" : "Pick Up Notes"})
merged_items_df.to_csv('merged_item_columns.csv', index=True)

In [ ]:
# Clean up data and upload to a database